#**Calculadora de Tamanho de Amostra para Teste A/B**

Como vimos em aula, definir o tamanho da amostra é um passo bem importante para realizarmos nosso teste A/B.

Neste notebook traz funções para auxiliar vocês nesse trabalho. Mas vale ressalvar que essas funções calculam tamanho de amostra quando nossa métrica de interesse é uma proporção. Por exemplo, percentual de conversão, percentual de churn... Mas esse tipo de métrica é a mais usada nos testes A/B.

## **Observações importantes :**
- Precisamos de pelo menos dois grupos quando fazemos um teste A/B:
o controle (que é a vida como ela sempre foi) e o alternativo (onde você faz alguma modificação que é o que você quer medir, na prática)
- O cálculo amostral é pra que você encontre o tamanho mínimo necessário para conseguir caracterizar uma população.
- Ou seja, como o controle e a variante vem da mesma população, o número mínimo (tamanho de amostra) é o mesmo, então você pode utilizar o mesmo tamanho de amostra que encontrar para o grupo controle e para o grupo variante, o valor encontrado é o recomendado para cada grupo.
- Escolha o erro a ser utilizado ele pode ser tanto um erro absoluto quanto um erro relativo.

### **Importações**

In [3]:
from math import ceil
from scipy.stats import norm

## **Cálculo utilizando o erro relativo**

O código abaixo é usado para calcular o tamanho da amostra necessário para um teste A/B, dado um erro máximo permitido (max_lift_error), um nível de confiança (confidence_level) e uma taxa de controle (control_rate). A taxa de controle é a taxa de sucesso esperada no grupo de controle, enquanto o erro máximo permitido é a diferença máxima aceitável entre a taxa de controle e a taxa de sucesso no grupo de teste (é o lift minimo que queremos pegar como significativo). O nível de confiança é a probabilidade de que a diferença observada entre os grupos de teste e controle seja real e não devida ao acaso. Recomendo a utilização de no mínimo 95%.

Exemplo de control_rate = Média da taxa de conversão atual.

In [21]:
def sample_size_relative_error(max_lift_error, confidence_level, control_rate):
    # Calculate the alternative rate by adding the maximum allowed error to the control rate
    alternative_rate = (1 + max_lift_error) * control_rate

    # Calculate the average of the control and alternative rates
    avg_rate = (control_rate + alternative_rate) / 2

    # Calculate the significance level from the confidence level
    significance_level = 1-(1-confidence_level)/2

    # Obtain the z-quantile corresponding to the significance level
    z_quantile = norm.ppf(significance_level)

    # Calculate the first temporary term for the sample size formula
    term1 = avg_rate * (1 - avg_rate)

    # Calculate the second temporary term for the sample size formula
    term2 = (control_rate - alternative_rate) ** 2

    # Calculate the required sample size to achieve the maximum allowed error with the desired confidence level
    sample_size = (z_quantile ** 2) * 2 * term1 / term2

    # Return the sample size
    return ceil(sample_size)


### Exemplo

In [22]:
control_rate = 0.08
confidence_level = 0.95
max_lift_error = 0.05

sample_size_relative_error(max_lift_error, confidence_level, control_rate)

36147

## **Cálculo utilizando o erro relativo e definindo o poder do teste**

- baseline_conversion_rate: A taxa de conversão base para o grupo de controle.
- minimum_detectable_effect: O tamanho mínimo do efeito detectável, expresso como uma porcentagem da taxa de conversão base.
- alpha: O nível de significância para o teste. O padrão é 0.05.
- beta: O poder estatístico desejado para o teste. O padrão é 0.2.

In [ ]:
def sample_size_calculator(baseline_conversion_rate, minimum_detectable_effect, alpha=0.05, beta=0.2):

    # Calcula os desvios normais padrão para os níveis alfa e beta desejados
    z_alpha = norm.ppf(1 - alpha / 2)
    z_beta = norm.ppf(1 - beta)

    # Calcula o desvio padrão agrupado
    p1 = baseline_conversion_rate
    p2 = baseline_conversion_rate * (1 + minimum_detectable_effect)
    pooled_sd = ((p1 * (1 - p1) + p2 * (1 - p2)) / 2) ** 0.5

    # Calcula o tamanho mínimo da amostra
    sample_size = ((z_alpha + z_beta) * 2 * pooled_sd * 2) / (p1 - p2) ** 2
    sample_size = ceil(sample_size)

    return sample_size

### Exemplo

In [ ]:
baseline_conversion_rate = 0.08
minimum_detectable_effect = 0.05

sample_size_calculator(baseline_conversion_rate, minimum_detectable_effect)

192159